In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append("/home/shizhenkun/codebase/BioUniprot/tools/")
import funclib
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn import linear_model, datasets
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

In [2]:
def load_file(path):
    train = pd.read_hdf(path + 'train.h5', key='data')
    train.ec_number = train.ec_number.apply(lambda x: x.replace(',', ''))
    test = pd.read_hdf(path + 'test.h5', key='data')
    test.ec_number = test.ec_number.apply(lambda x: x.replace(',', ''))
    head = funclib.table_head + ['f'+str(i) for i in range(1, 1901)]
    train.columns = head
    test.columns = head
    return train, test

def xgmain(X_train_std, Y_train, X_test_std, Y_test):
    model = XGBClassifier(objective='binary:logistic', random_state=42, use_label_encoder=False, n_jobs=-2, eval_metric='mlogloss')
    model.fit(X_train_std, Y_train.ravel())
    predict = model.predict(X_test_std)
    predictprob = model.predict_proba(X_test_std)
    groundtruth = Y_test
    return groundtruth, predict, predictprob[:,1]


def caculate_metrics(groundtruth, predict, predictprob):
    acc = metrics.accuracy_score(groundtruth, predict)
    precision = metrics.precision_score(groundtruth, predict, zero_division=1 )
    recall = metrics.recall_score(groundtruth, predict)
    f1 = metrics.f1_score(groundtruth, predict)
    auroc = metrics.roc_auc_score(groundtruth, predictprob)
    auprc = metrics.average_precision_score(groundtruth, predictprob)
    tn, fp, fn, tp = metrics.confusion_matrix(groundtruth, predict).ravel()

    npv = tn/(fn+tn+1.4E-45)
    print('baslineName', '\t', 'accuracy','\t', 'precision(PPV) \t NPV \t\t', 'recall','\t', 'f1', '\t\t', 'auroc','\t\t', 'auprc', '\t\t confusion Matrix')
    print('xgboost', '\t%f' %acc,'\t%f'% precision,'\t\t%f'%npv,'\t%f'% recall,'\t%f'% f1, '\t%f'% auroc,'\t%f'% auprc, '\t', 'tp:',tp,'fp:',fp,'fn:',fn,'tn:',tn)

In [3]:
ori_file_dir = '/home/shizhenkun/codebase/BioUniprot/tasks/task1/data/'
train, test = load_file(ori_file_dir)

In [4]:
res_data=funclib.getblast(train,test)

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1


In [ ]:
res_data=funclib.getblast(train,test)
funclib.run_integrated(res_data, train, test)

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1
baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix
lr 	 0.948454  	0.941863 		 0.954155  	0.946725	 0.944288		 	 	 	 tp:19441  fp:1200  fn:1094  tn:22769
xg 	 0.952431  	0.947691 		 0.956505  	0.949306	 0.948498		 	 	 	 tp:19494  fp:1076  fn:1041  tn:22893
dt 	 0.926703  	0.905412 		 0.946252  	0.939274	 0.922033		 	 	 	 tp:19288  fp:2015  fn:1247  tn:21954
rf 	 0.952746  	0.954214 		 0.951516  	0.942829	 0.948487		 	 	 	 tp:19361  fp:929  fn:1174  tn:23040


In [ ]:
# xg_res= pd.DataFrame([groundtruth_xg, predict_xg, predictprob_xg])
# xg_res = xg_res.T
# xg_res.rename(columns={0:'ground_truth', 1: 'predict', 2:'pred_prob'}, inplace=True)
# xg_res.to_excel('./results/xg.xlsx', index=0, header=1)